In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

In [20]:
data_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=60),
    # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder("MRI Scans/Training", transform=data_transform)
test_dataset = datasets.ImageFolder("MRI Scans/Testing", transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [31]:
resnet = models.resnet50(pretrained=True)
googlenet = models.googlenet(pretrained=True)

# for param in resnet.parameters():
#     param.requires_grad = False
# for param in googlenet.parameters():
#     param.requires_grad = False

In [32]:
# resnet.fc = nn.Linear(resnet.fc.in_features, 4)

# googlenet.fc = nn.Linear(googlenet.fc.in_features, 4)

resnet.fc = nn.Sequential(
    nn.Linear(resnet.fc.in_features, 2048),
    nn.LeakyReLU(negative_slope=0.01, inplace=True),
    nn.Dropout(p=0.4),
    nn.Linear(2048, 2048),
    nn.LeakyReLU(negative_slope=0.01, inplace=True),
    nn.Dropout(p=0.4),
    nn.Linear(2048, 4)
)

googlenet.fc = nn.Sequential(
    nn.Linear(googlenet.fc.in_features, 2048),
    nn.LeakyReLU(negative_slope=0.01, inplace=True),
    nn.Dropout(p=0.4),
    nn.Linear(2048, 2048),
    nn.LeakyReLU(negative_slope=0.01, inplace=True),
    nn.Dropout(p=0.4),
    nn.Linear(2048, 4)
)

In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

resnet = resnet.to(device)
googlenet = googlenet.to(device)

Using device: cuda


In [34]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(list(resnet.fc.parameters()) + list(googlenet.fc.parameters()), lr=5e-5, weight_decay=1e-4)

In [35]:
for epoch in range(30):
    resnet.train()
    googlenet.train()
    total_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        resnet_out = resnet(images)
        googlenet_out = googlenet(images)

        # Ensemble: average outputs
        outputs = (resnet_out + googlenet_out) / 2

        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}: Loss = {total_loss/len(train_loader):.4f}")

Epoch 1: Loss = 1.1074
Epoch 2: Loss = 0.6610
Epoch 3: Loss = 0.5664
Epoch 4: Loss = 0.5203
Epoch 5: Loss = 0.4925
Epoch 6: Loss = 0.4476
Epoch 7: Loss = 0.4233
Epoch 8: Loss = 0.4323
Epoch 9: Loss = 0.4131
Epoch 10: Loss = 0.4155
Epoch 11: Loss = 0.3972
Epoch 12: Loss = 0.3761
Epoch 13: Loss = 0.3766
Epoch 14: Loss = 0.3829
Epoch 15: Loss = 0.3740
Epoch 16: Loss = 0.3495
Epoch 17: Loss = 0.3673
Epoch 18: Loss = 0.3689
Epoch 19: Loss = 0.3710
Epoch 20: Loss = 0.3579
Epoch 21: Loss = 0.3343
Epoch 22: Loss = 0.3355
Epoch 23: Loss = 0.3396
Epoch 24: Loss = 0.3382
Epoch 25: Loss = 0.3426
Epoch 26: Loss = 0.3412
Epoch 27: Loss = 0.3063
Epoch 28: Loss = 0.3250
Epoch 29: Loss = 0.3257
Epoch 30: Loss = 0.3230


In [36]:
resnet.eval()
googlenet.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        resnet_out = resnet(images)
        googlenet_out = googlenet(images)
        outputs = (resnet_out + googlenet_out) / 2
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total * 100
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 62.94%


In [37]:
resnet.eval()
googlenet.eval()
correct_train = 0
total_train = 0

with torch.no_grad():
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        resnet_out = resnet(images)
        googlenet_out = googlenet(images)
        outputs = (resnet_out + googlenet_out) / 2
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

train_accuracy = correct_train / total_train * 100
print(f"Training Accuracy: {train_accuracy:.2f}%")

Training Accuracy: 86.52%
